# Tutorial: Upload and download blobs from a container
This tutorial uses the latest version of azure-blob-storage(v12.14.1) and demonstrate how to upload and download blobs. 

In [ ]:
# set connection string of the storage in the environment 
import os
os.environ['AZURE_STORAGE_CONNECTION_STRING']="your storage connection string"
container_name = 'data'
blob_folder = os.path.join(os.getcwd(), container_name+"-blobs")
os.makedirs(blob_folder, exist_ok=True)

## Download the files from a container 

In [ ]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

try:
    print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")
    connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

    print("\nListing blobs...")

    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    container_client = blob_service_client.get_container_client(container_name)
    # List the blobs in the container
    blob_list = container_client.list_blobs()

    for blob in blob_list:
        print("\t" + blob.name)
        download_file_path = os.path.join(blob_folder, blob.name)
        with open(download_file_path, "wb") as download_file:
            download_file.write(container_client.download_blob(blob.name).readall())
            print( blob.name + " downloaded")

except Exception as ex:
    print('Exception:')
    print(ex)

# Upload a file into a container

In [ ]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
try:
    print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")
    connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')

    file_name = 'A file name'
    upload_file_path = os.path.join(os.getcwd(), file_name)
    
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=file_name)

    print("\nUploading to Azure Storage as blob:\n\t" + file_name)

    # Upload the created file
    with open(upload_file_path, "rb") as data:
        blob_client.upload_blob(data)
    
except Exception as ex:
    print('Exception:')
    print(ex)

## Download Parquet files with nested folders from a container

In [ ]:
import os, uuid
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__

try:
    print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")
    connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
    
    blob_service_client = BlobServiceClient.from_connection_string(connect_str)
    container_client = blob_service_client.get_container_client(container_name)
    # choose a specific blob for downloading 
    blob_list = container_client.list_blobs(name_starts_with="path of a specific blob")

    blob_path = "path of the specific blob"

    for blob in blob_list:
        # Don't want to create empty unnecessary nested folders, therfore replace the path with a null string
        blob_name = blob.name.replace(blob_path, "")
        if blob_name.endswith(".parquet"):
            #download file
            download_file_path = os.path.join(blob_folder, blob_name)
            isExist = os.path.exists(download_file_path)
            if not isExist:
                with open(download_file_path, "wb") as download_file:
                    download_file.write(container_client.download_blob(blob.name).readall())
                    print(download_file_path + " downloaded")
            else:
                print(download_file_path + " already exists" + "   No Need to download!")
        else:
            #create a folder
            dir_path = os.path.join(blob_folder, blob_name)
            os.makedirs(dir_path, exist_ok=True)

except Exception as ex:
    print('Exception:')
    print(ex)

## Upload nested folders into a container

In [ ]:
import os
from azure.storage.blob import BlobServiceClient
import argparse
import glob

connect_str = os.getenv('AZURE_STORAGE_CONNECTION_STRING')
# Instantiate a BlobServiceClient using a connection string
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

# Instantiate a ContainerClient
container_client = blob_service_client.get_container_client(container_name) 

# Create new Container
try:
    container_client.create_container()
except Exception as err:
    print("WARNING: ", err)
    pass

data_path = os.path.join(os.getcwd(), 'folder-name') # its need the mnt path of your folder, otherwise you will get an authorization error!

for filename in glob.iglob(os.path.join(data_path, '**', '*'), recursive=True):
    if os.path.isfile(filename):
        # Upload a blob to the container
        with open(filename, "rb") as data:
            try:
                print('Uploading ', filename)
                container_client.upload_blob(name=filename, data=data)
            except Exception as err:
                print("WARNING: ", err)
                pass

# Summary
This tutorials explain how can we download/upload files.
[Azure storage](https://learn.microsoft.com/en-us/azure/storage/blobs/storage-quickstart-blobs-python?tabs=managed-identity%2Croles-azure-portal%2Csign-in-azure-cli)
